In [2]:
from confluent_kafka import Consumer, KafkaException, KafkaError
import psycopg2
import json
import random  # Importer la bibliothèque random

# Configuration du consommateur Kafka
conf = {
    'bootstrap.servers': 'kafka:9092',  # Adresse du broker Kafka
    'group.id': 'my_group',              # ID du groupe de consommateurs
    'auto.offset.reset': 'earliest',     # Commencer à lire à partir du début du topic
}

# Création d'une instance de consommateur
print("Création du consommateur Kafka...")
consumer = Consumer(conf)

# Inscription au topic
topic = 'mastodon-toots'  # Remplacez par le nom de votre topic
print(f"Inscription au topic: {topic}...")
consumer.subscribe([topic])

# Configuration de la connexion à PostgreSQL
print("Connexion à PostgreSQL...")
conn = psycopg2.connect(
    host="postgres",  # Changez ceci si nécessaire
    database="DB_Mastodon",
    user="fadi",
    password="fadi"
)
cursor = conn.cursor()

# Création de la table si elle n'existe pas déjà
create_table_query = '''
CREATE TABLE IF NOT EXISTS Toots_BRONZ (
    id BIGINT PRIMARY KEY,
    username TEXT NOT NULL,
    display_name TEXT NOT NULL,
    content TEXT NOT NULL,
    favourites_count INT NOT NULL,
    reblogs_count INT NOT NULL,
    replies_count INT NOT NULL
);
'''
cursor.execute(create_table_query)
conn.commit()

try:
    while True:
        # Lire les messages du topic
        msg = consumer.poll(1.0)  # Timeout de 1 seconde
        
        if msg is None:
            continue  # Pas de message reçu
        if msg.error():
            # Gestion des erreurs
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                continue
        
        # Traiter le message
        message_value = msg.value().decode('utf-8')

        # Convertir le message JSON en dictionnaire
        try:
            message_dict = json.loads(message_value)
            
            # Vérifier que les champs nécessaires sont présents
            required_fields = ['id', 'username', 'display_name', 'content']
            if not all(field in message_dict for field in required_fields):
                continue

            # Générer des valeurs aléatoires pour 'favourites_count', 'reblogs_count', et 'replies_count'
            favourites_count = random.randint(0, 100)  # Nombre aléatoire entre 0 et 100
            reblogs_count = random.randint(0, 50)      # Nombre aléatoire entre 0 et 50
            replies_count = random.randint(0, 30)      # Nombre aléatoire entre 0 et 30

            # Insertion du message dans PostgreSQL
            insert_query = '''INSERT INTO Toots_BRONZ (id, username, display_name, content, favourites_count, reblogs_count, replies_count) 
                              VALUES (%s, %s, %s, %s, %s, %s, %s) 
                              ON CONFLICT (id) DO NOTHING;'''  # Ignorer les doublons basés sur l'ID
            cursor.execute(insert_query, (
                message_dict['id'],
                message_dict['username'],
                message_dict['display_name'],
                message_dict['content'],
                favourites_count,   # Valeur aléatoire pour favourites_count
                reblogs_count,      # Valeur aléatoire pour reblogs_count
                replies_count       # Valeur aléatoire pour replies_count
            ))
            conn.commit()  # Valider la transaction

        except json.JSONDecodeError:
            continue
        except Exception:
            continue

except KeyboardInterrupt:
    pass
finally:
    # Compter le nombre total d'entrées dans la table 'toots'
    print("Calcul du nombre total d'entrées dans la table 'toots'...")
    cursor.execute("SELECT COUNT(*) FROM toots_BRONZ;")
    count = cursor.fetchone()[0]  # Récupérer le résultat du COUNT
    print(f"Nombre total d'entrées dans la table 'toots_BRONZ' : {count}")

    # Fermer le consommateur et la connexion à PostgreSQL
    consumer.close()
    cursor.close()
    conn.close()


Création du consommateur Kafka...
Inscription au topic: mastodon-toots...
Connexion à PostgreSQL...
Calcul du nombre total d'entrées dans la table 'toots'...
Nombre total d'entrées dans la table 'toots_BRONZ' : 30687
